# Federated PyTorch TinyImageNet Tutorial

This notebook is an example of Transfer Learning 

Custom DataLoader is used with OpenFL Python API

In [ ]:
#Install dependencies if not already installed
!pip install torch torchvision peft transformers sentencepiece huggingface_hub accelerate datasets  evaluate  seqeval
%load_ext autoreload
%autoreload 2

In [1]:
from typing import Any, Mapping
import numpy as np
import openfl.native as fx
import torch
import torch as pt
from accelerate import Accelerator
from datasets import Dataset, load_dataset, load_metric
from openfl.federated import PyTorchTaskRunner, TaskRunner
from openfl.federated.task.runner_pt import change_tags
from openfl.utilities import Metric, TensorKey
from openfl.utilities.data_splitters import EqualNumPyDataSplitter
from peft import LoraConfig, TaskType, get_peft_model
from peft.utils import get_peft_model_state_dict, set_peft_model_state_dict
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn

from transformers import (AutoConfig, AutoModelForSequenceClassification,
                          AutoTokenizer, DataCollatorWithPadding)

After importing the required packages, the next step is setting up our openfl workspace. To do this, simply run the `fx.init()` command as follows:

In [2]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist')

Creating Workspace Directories
Creating Workspace Templates
  Using cached torch-1.13.1-cp38-cp38-manylinux1_x86_64.whl (887.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0
    Uninstalling torch-2.1.0:
      Successfully uninstalled torch-2.1.0

New workspace directory structure:
workspace
├── logs
│   └── cnn_mnist
│       ├── events.out.tfevents.1695850586.M50CYP2SBSTD.111429.0
│       ├── events.out.tfevents.1695942084.M50CYP2SBSTD.4924.0
│       ├── events.out.tfevents.1695849809.M50CYP2SBSTD.107313.0
│       ├── events.out.tfevents.1695850472.M50CYP2SBSTD.110437.0
│       ├── events.out.tfevents.1695942744.M50CYP2SBSTD.15635.0
│       ├── events.out.tfevents.1696008244.M50CYP2SBSTD.98097.0
│       ├── events.out.tfevents.1695850981.M50CYP2SBSTD.114740.0
│       ├── events.out.tfevents.1695939101.M50CYP2SBSTD.143673.0
│       ├── events.out.tfevents.1695850850.M50CYP2SBSTD.113094.0
│       ├── events.out.tfevents.1695850404.M50CYP2SBSTD.109391.0
│ 

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

#### Download the data

In [3]:
def get_glue_mrpc_dataset(tokenizer):
    dataset = load_dataset("glue", "mrpc")

    def tokenize_function(examples):
        # max_length=None => use the model max length (it's actually the default)
        outputs = tokenizer(
            examples["sentence1"],
            examples["sentence2"],
            truncation=True,
            max_length=None,
        )
        return outputs

    tokenized_datasets = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["idx", "sentence1", "sentence2"],
    )
    tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
    tokenized_datasets.set_format("torch")
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")
    return data_collator, tokenized_datasets

base_model_name = "roberta-large"
padding_side = "right"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
data_collator, tokenized_datasets = get_glue_mrpc_dataset(tokenizer)

#### Describe the dataset

In [4]:
class GlueMrpc(Dataset):
    """
    Has 5.8k pairs of sentences with annotations if the two sentences are equivalent
    """    
    def get_shape(self):
        
        if not hasattr(self, 'saved_shape'):
            self.saved_shape = max([len(i) for i in self.data['input_ids']])
        return self.saved_shape

train_set = GlueMrpc.from_dict(tokenized_datasets['train'].to_dict())
valid_set = GlueMrpc.from_dict(tokenized_datasets['test'].to_dict())


#### Implement Federated dataset
We have to implement `split` method

In [5]:
class GlueMrpcFederatedDataset(DataLoader):
    def __init__(self, train_set, valid_set, batch_size, data_collator=None):
        self.data_splitter = EqualNumPyDataSplitter()
        if isinstance(train_set,Dataset):
            self.train_set = GlueMrpc.from_dict(train_set.to_dict())
        else:
            self.train_set = train_set
            
        if isinstance(valid_set,Dataset):
            self.valid_set = GlueMrpc.from_dict(valid_set.to_dict())
        else:
            self.valid_set = valid_set            
            
        self.batch_size = batch_size
        self.data_collator = data_collator
    
    def split(self, num_collaborators):
        train_split = self.data_splitter.split(self.train_set, num_collaborators)
        valid_split = self.data_splitter.split(self.valid_set, num_collaborators)
        return [
            GlueMrpcFederatedDataset(
                self.train_set.select(train_split[i]),
                self.valid_set.select(valid_split[i]),
                self.batch_size
            )
            for i in range(num_collaborators)
        ]
    
    def get_feature_shape(self):
        return self.train_set.get_shape()
    
    def get_train_loader(self, num_batches=None):
        return DataLoader(self.train_set, batch_size=self.batch_size, collate_fn=data_collator)
    
    def get_valid_loader(self):
        return DataLoader(self.valid_set, collate_fn=data_collator)
    
    def get_train_data_size(self):
        return len(self.train_set)
    
    def get_valid_data_size(self):
        return len(self.valid_set)
    
fl_data = GlueMrpcFederatedDataset(train_set, valid_set, batch_size=32)

#### Define model

In [6]:
class LLMTaskRunner(PyTorchTaskRunner):
    def __init__(self, base_model_name, data_loader, device=None, metric=None, **kwargs):
        kwargs['data_loader'] = data_loader
        super().__init__(device, **kwargs)
        self.base_model_name = base_model_name
        self.metric = metric
        self._init_model()
        self._init_optimizer()
        
    def _init_model(self):
        model = AutoModelForSequenceClassification.from_pretrained(
    self.base_model_name, return_dict=True)
        peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, r=16, lora_alpha=16, lora_dropout=0.1, bias="all")
        self.model = get_peft_model(model, peft_config)
        
    def _init_optimizer(self):
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
                {
                    "params": [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
                    "weight_decay": 0.01,
                },
                {
                    "params": [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)],
                    "weight_decay": 0.0,
                },
            ]
        self.optimizer = AdamW(optimizer_grouped_parameters, lr=0.01)
        
        self.training_round_completed = False
        self.initialize_tensorkeys_for_functions()
    
    def state_dict(self):
        return get_peft_model_state_dict(self.model)
    
    def load_state_dict(self, state_dict: Mapping[str, Any], strict: bool = True):
        return set_peft_model_state_dict(
                self.model, state_dict
            )
    
    def validate(self, col_name, round_num, input_tensor_dict,
                 use_tqdm=False, **kwargs):
        """Validate.

        Run validation of the model on the local data.

        Args:
            col_name:            Name of the collaborator
            round_num:           What round is it
            input_tensor_dict:   Required input tensors (for model)
            use_tqdm (bool):     Use tqdm to print a progress bar (Default=True)

        Returns:
            global_output_dict:  Tensors to send back to the aggregator
            local_output_dict:   Tensors to maintain in the local TensorDB

        """
        self.rebuild_model(round_num, input_tensor_dict, validation=True)
        self.model.eval()
        self.model.to(self.device)
        val_score = 0
        total_samples = 0

        loader = self.data_loader.get_valid_loader()
        if use_tqdm:
            loader = tqdm(loader, desc='validate')

        with pt.no_grad():
            for sample in loader:
                samples = sample['input_ids'].shape[0]
                total_samples += samples
                output = self.model(**sample)
                # get the index of the max log-probability
                logits = output.logits
                predictions = torch.argmax(logits, dim=-1)
                metric.add_batch(predictions=predictions, references=sample['labels'])
        val_score = metric.compute()['accuracy']

        origin = col_name
        suffix = 'validate'
        if kwargs['apply'] == 'local':
            suffix += '_local'
        else:
            suffix += '_agg'
        tags = ('metric',)
        tags = change_tags(tags, add_field=suffix)
        # TODO figure out a better way to pass in metric for this pytorch
        #  validate function
        output_tensor_dict = {
            TensorKey('acc', origin, round_num, True, tags):
                np.array(val_score)
        }

        # Empty list represents metrics that should only be stored locally
        return output_tensor_dict, {}

    def train_epoch(self, batch_generator) -> Metric:
        """Train single epoch.

        Override this function in order to use custom training.

        Args:
            batch_generator: Train dataset batch generator. Yields (samples, targets) tuples of
            size = `self.data_loader.batch_size`.
        Returns:
            Metric: An object containing name and np.ndarray value.
        """
        losses = []
        for sample in batch_generator:
            self.optimizer.zero_grad()
            output = self.model(**sample)
            loss = output.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(),1.0)
            self.model.zero_grad()
            losses.append(loss.detach().cpu().numpy())
        loss = np.mean(losses)
        if self.model.config.problem_type == "regression":
            loss_fct = MSELoss()
        elif self.model.config.problem_type == "single_label_classification":
            loss_fct = CrossEntropyLoss()
        elif self.model.config.problem_type == "multi_label_classification":
            loss_fct = BCEWithLogitsLoss()
        return Metric(name=loss_fct._get_name(), value=np.array(loss))
        
        
    def save_native(self, filepath, model_state_dict_key='model_state_dict',
                    optimizer_state_dict_key='optimizer_state_dict', **kwargs):
        """
        Save model and optimizer states in a picked file specified by the \
        filepath. model_/optimizer_state_dicts are stored in the keys provided. \
        Uses pt.save().

        Args:
            filepath (string)                 : Path to pickle file to be
                                                created by pt.save().
            model_state_dict_key (string)     : key for model state dict
                                                in pickled file.
            optimizer_state_dict_key (string) : key for optimizer state
                                                dict in picked file.
            kwargs                            : unused

        Returns:
            None
        """
        pickle_dict = {
            model_state_dict_key: get_peft_model_state_dict(self.model),
            optimizer_state_dict_key: self.optimizer.state_dict()
        }
        pt.save(pickle_dict, filepath)

In [7]:
num_collaborators = 2
metric = load_metric('glue', "mrpc")
collaborator_models = [
            LLMTaskRunner(
                base_model_name,
                data_loader=data_slice,
                metric=metric
            )
            for data_slice in fl_data.split(num_collaborators)]
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

/tmp/ipykernel_159004/1723172838.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', "mrpc")
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[08:48:31] WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=932122;file:///home/oamontoy/workspace/sec-openfl/openfl/utilities/utils.py\utils.py]8;;\:]8;id=685149;file:///home/oamontoy/workspace/sec-openfl/openfl/utilities/utils.py#172\172]8;;\

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[08:48:34] WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=38894;file:///home/oamontoy/workspace/sec-openfl/openfl/utilities/utils.py\utils.py]8;;\:]8;id=682120;file:///home/oamontoy/workspace/sec-openfl/openfl/utilities/utils.py#172\172]8;;\

In [8]:
#Original TinyImageNet dataset
print(f'Original training data size: {len(fl_data.train_set)}')
print(f'Original validation data size: {len(fl_data.valid_set)}\n')

#Collaborator one's data
for i, model in enumerate(collaborator_models):
    print(f'Collaborator {i}\'s training data size: {len(model.data_loader.train_set)}')
    print(f'Collaborator {i}\'s validation data size: {len(model.data_loader.valid_set)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

Original training data size: 3668
Original validation data size: 1725

Collaborator 0's training data size: 1834
Collaborator 0's validation data size: 863

Collaborator 1's training data size: 1834
Collaborator 1's validation data size: 862



In [9]:
#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(collaborators,{'aggregator.settings.rounds_to_train':3})

           INFO     Updating aggregator.settings.rounds_to_train to 3...                                                                           ]8;id=22181;file:///home/oamontoy/workspace/sec-openfl/openfl/native/native.py\native.py]8;;\:]8;id=265323;file:///home/oamontoy/workspace/sec-openfl/openfl/native/native.py#102\102]8;;\

           INFO     FL-Plan hash is 86b08340e96ba9e485169da1f860ea968811d1bf2e6867774fae4398426dd33c6ae56ca202002d393e3a4d91f946c1bc                 ]8;id=965130;file:///home/oamontoy/workspace/sec-openfl/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=969631;file:///home/oamontoy/workspace/sec-openfl/openfl/federated/plan/plan.py#235\235]8;;\

           INFO     Building 🡆 Object NoCompressionPipeline from openfl.pipelines Module.                                                            ]8;id=418157;file:///home/oamontoy/workspace/sec-openfl/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=469081;file:///home/oamontoy/workspace/sec-openfl/openfl/federated/plan/plan.py#171\171]8;;\

           INFO     Creating Initial Weights File    🠆 save/torch_cnn_mnist_init.pbuf                                                              ]8;id=949808;file:///home/oamontoy/workspace/sec-openfl/openfl/native/native.py\native.py]8;;\:]8;id=806002;file:///home/oamontoy/workspace/sec-openfl/openfl/native/native.py#277\277]8;;\

           INFO     Starting Experiment...                                                                                                         ]8;id=768304;file:///home/oamontoy/workspace/sec-openfl/openfl/native/native.py\native.py]8;;\:]8;id=820559;file:///home/oamontoy/workspace/sec-openfl/openfl/native/native.py#281\281]8;;\

...

           INFO     Collaborator one is sending task results for aggregated_model_validation, round 0                                          ]8;id=921737;file:///home/oamontoy/workspace/sec-openfl/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=943670;file:///home/oamontoy/workspace/sec-openfl/openfl/component/aggregator/aggregator.py#520\520]8;;\

           METRIC   Round 0, collaborator validate_agg aggregated_model_validation result acc:      0.673233                                   ]8;id=778930;file:///home/oamontoy/workspace/sec-openfl/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=605180;file:///home/oamontoy/workspace/sec-openfl/openfl/component/aggregator/aggregator.py#559\559]8;;\

           INFO     Using TaskRunner subclassing API                                                                                         ]8;id=239383;file:///home/oamontoy/workspace/sec-openfl/openfl/component/collaborator/collaborator.py\collaborator.py]8;;\:]8;id=814426;file:///home/oamontoy/workspace/sec-openfl/openfl/component/collaborator/collaborator.py#253\253]8;;\

           INFO     Run 0 epoch of 0 round                                                                                                      ]8;id=118270;file:///home/oamontoy/workspace/sec-openfl/openfl/federated/task/runner_pt.py\runner_pt.py]8;;\:]8;id=848215;file:///home/oamontoy/workspace/sec-openfl/openfl/federated/task/runner_pt.py#155\155]8;;\

AttributeError: 'RobertaForSequenceClassification' object has no attribute 'step'

In [ ]:
#Save final model
final_fl_model.save_native('final_model.pth')